# MLP - `Auto_MPG_data` Dataset

In [50]:
# ! gdown --id 1qiUDDoYyRLBiKOoYWdFl_5WByHE8Cugu -O "data/Auto_MPG_data.csv"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
random_sate = 59
np.random.seed(random_sate)
torch.manual_seed(random_sate)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_sate)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
df = pd.read_csv("data/Auto_MPG_data.csv")
df.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
0,18.0,8,307.0,130.0,3504.0,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,0,0,1


### Data preprocessing

In [5]:
X = df.iloc[:, 1:].values
y = df["MPG"].values

X.shape, y.shape

((392, 9), (392,))

In [6]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=random_sate,
    shuffle=True
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=0.125,
    random_state=random_sate,
    shuffle=True
)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((273, 9), (79, 9), (40, 9), (273,), (79,), (40,))

In [7]:
normalizer = StandardScaler()

X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### DataLoader

In [8]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [9]:
batch_size = 32

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### R2 score

In [10]:
def r_squared(y, y_pred):
    y = torch.Tensor(y).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y)
    ss_tot = torch.sum((y - mean_true) ** 2)
    ss_res = torch.sum((y - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

### MLP

In [27]:
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
    
    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        out = self.output(x)
        return out.squeeze(1)

In [28]:
model = MLP(input_dims=X_train.shape[1], hidden_dims=64, output_dims=1).to(device)
model

MLP(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
)

### Loss and Optimizer

In [29]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

### Train

In [30]:
epochs = 100
train_losses, val_losses = [], []
train_r2, val_r2 = [], []

for epoch in range(epochs):
    train_loss = 0.0
    train_target, train_predict = [], []
    val_target, val_predict = [], []
    
    model.train()
    
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(X_samples)
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()

        train_loss += loss.item()
        
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))
    
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            
            outputs = model(X_samples)
            loss = criterion(outputs, y_samples)
            
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))
    print(f"\n Epoch {epoch + 1}: \tTraining loss: {train_loss:.3f} \tValidation loss: {val_loss:.3f}")


 Epoch 1: 	Training loss: 396.203 	Validation loss: 172.711

 Epoch 2: 	Training loss: 149.594 	Validation loss: 44.465

 Epoch 3: 	Training loss: 121.731 	Validation loss: 12.565

 Epoch 4: 	Training loss: 18.505 	Validation loss: 280.319

 Epoch 5: 	Training loss: 163.226 	Validation loss: 34.213

 Epoch 6: 	Training loss: 20.306 	Validation loss: 11.482

 Epoch 7: 	Training loss: 12.106 	Validation loss: 11.638

 Epoch 8: 	Training loss: 13.820 	Validation loss: 16.028

 Epoch 9: 	Training loss: 13.846 	Validation loss: 7.095

 Epoch 10: 	Training loss: 15.788 	Validation loss: 9.309

 Epoch 11: 	Training loss: 11.958 	Validation loss: 14.595

 Epoch 12: 	Training loss: 10.753 	Validation loss: 7.530

 Epoch 13: 	Training loss: 7.541 	Validation loss: 10.285

 Epoch 14: 	Training loss: 14.035 	Validation loss: 7.154

 Epoch 15: 	Training loss: 7.222 	Validation loss: 6.249

 Epoch 16: 	Training loss: 6.965 	Validation loss: 5.568

 Epoch 17: 	Training loss: 10.797 	Validation loss:

### Evaluate

In [31]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device))
    test_set_r2 = r_squared(y_test, y_pred)
    print("Evaluation on test set")
    print(f"R2: {test_set_r2}")

Evaluation on test set
R2: 0.8743938207626343


### Question 6

In [20]:
X_train = torch.tensor([[5.1], [6.0], [5.7]], dtype=torch.float32)
y_train = torch.tensor([[0], [1], [1]], dtype=torch.float32)

class FlowersMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(1, 2)
        self.relu = nn.ReLU()
        self.output = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
        with torch.no_grad():
            self.hidden.weight = nn.Parameter(torch.tensor([[0.5], [-0.5]], dtype=torch.float32))
            self.hidden.bias = nn.Parameter(torch.tensor([1.0, -1.0], dtype=torch.float32))
            self.output.weight = nn.Parameter(torch.tensor([[0.3, -0.2]], dtype=torch.float32))
            self.output.bias = nn.Parameter(torch.tensor([0.5], dtype=torch.float32))
    
    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)

model = FlowersMLP()

x_test = torch.tensor([[5.1]])
with torch.no_grad():
    output = model(x_test)
    prediction = 1 if output.item() > 0.5 else 0

print("Question 6:")
print(f"x = 5.1 - Probability: {output.item():.3f} - Predicted class: {prediction}")

Question 6:
x = 5.1 - Probability: 0.827 - Predicted class: 1


### Question 7

In [21]:
X_train = torch.tensor([[50], [60], [70]], dtype=torch.float32)
y_train = torch.tensor([[100], [120], [140]], dtype=torch.float32)

class HousePriceMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(1, 2)
        self.relu = nn.ReLU()
        self.output = nn.Linear(2, 1)
        
        with torch.no_grad():
            self.hidden.weight = nn.Parameter(torch.tensor([[0.5], [-0.5]], dtype=torch.float32))
            self.hidden.bias = nn.Parameter(torch.tensor([1.0, -1.0], dtype=torch.float32))
            self.output.weight = nn.Parameter(torch.tensor([[1.5, 2.0]], dtype=torch.float32))
            self.output.bias = nn.Parameter(torch.tensor([0.0], dtype=torch.float32))
            
    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        return self.output(x)

model = HousePriceMLP()
x_input = torch.tensor([[50.0]])
with torch.no_grad():
    predicted_price = model(x_input)

print("Question 7:")
print(f"x = 50: {predicted_price.item():.2f}")

Question 7:
x = 50: 39.00


### Question 8.1

In [26]:
print("Question 8.1:")

class MLP81(nn.Module):
    def __init__(self, input_dims, output_dims):
        super().__init__()
        self.linear = nn.Linear(input_dims, output_dims)
    
    def forward(self, x):
        return self.linear(x).squeeze(1)

model = MLP81(X_train.shape[1], 1).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

epochs = 100
train_losses, val_losses = [], []
train_r2, val_r2 = [], []

for epoch in range(epochs):
    train_loss = 0.0
    train_target, train_predict = [], []
    val_target, val_predict = [], []
    
    model.train()
    
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(X_samples)
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()

        train_loss += loss.item()
        
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))
    
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            
            outputs = model(X_samples)
            loss = criterion(outputs, y_samples)
            
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))
    print(f"\n Epoch {epoch + 1}: \tTraining loss: {train_loss:.3f} \tValidation loss: {val_loss:.3f}")
    
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device))
    test_set_r2 = r_squared(y_test, y_pred)
    print("Evaluation on test set")
    print(f"R2: {test_set_r2}")

Question 8.1:

 Epoch 1: 	Training loss: 533.760 	Validation loss: 439.767

 Epoch 2: 	Training loss: 358.632 	Validation loss: 313.511

 Epoch 3: 	Training loss: 249.830 	Validation loss: 225.462

 Epoch 4: 	Training loss: 178.998 	Validation loss: 164.626

 Epoch 5: 	Training loss: 128.302 	Validation loss: 119.231

 Epoch 6: 	Training loss: 90.892 	Validation loss: 88.113

 Epoch 7: 	Training loss: 69.484 	Validation loss: 65.751

 Epoch 8: 	Training loss: 50.795 	Validation loss: 50.193

 Epoch 9: 	Training loss: 39.932 	Validation loss: 38.682

 Epoch 10: 	Training loss: 31.264 	Validation loss: 30.997

 Epoch 11: 	Training loss: 27.062 	Validation loss: 25.312

 Epoch 12: 	Training loss: 22.848 	Validation loss: 20.277

 Epoch 13: 	Training loss: 19.968 	Validation loss: 17.572

 Epoch 14: 	Training loss: 16.891 	Validation loss: 15.549

 Epoch 15: 	Training loss: 15.496 	Validation loss: 13.792

 Epoch 16: 	Training loss: 14.605 	Validation loss: 12.617

 Epoch 17: 	Training los

### Question 8.2

In [23]:
print("Question 8.2:")
class MLP82(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
    
    def forward(self, x):
        x = F.sigmoid(self.linear1(x))
        out = self.output(x)
        return out.squeeze(1)

model = MLP82(input_dims=X_train.shape[1], hidden_dims=64, output_dims=1).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

epochs = 100
train_losses, val_losses = [], []
train_r2, val_r2 = [], []

for epoch in range(epochs):
    train_loss = 0.0
    train_target, train_predict = [], []
    val_target, val_predict = [], []
    
    model.train()
    
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(X_samples)
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()

        train_loss += loss.item()
        
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))
    
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            
            outputs = model(X_samples)
            loss = criterion(outputs, y_samples)
            
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))
    print(f"\n Epoch {epoch + 1}: \tTraining loss: {train_loss:.3f} \tValidation loss: {val_loss:.3f}")
    
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device))
    test_set_r2 = r_squared(y_test, y_pred)
    print("Evaluation on test set")
    print(f"R2: {test_set_r2}")

Question 8.2:

 Epoch 1: 	Training loss: 146.833 	Validation loss: 16.647

 Epoch 2: 	Training loss: 20.054 	Validation loss: 10.036

 Epoch 3: 	Training loss: 15.585 	Validation loss: 9.003

 Epoch 4: 	Training loss: 14.345 	Validation loss: 8.059

 Epoch 5: 	Training loss: 13.605 	Validation loss: 8.060

 Epoch 6: 	Training loss: 13.013 	Validation loss: 7.782

 Epoch 7: 	Training loss: 12.795 	Validation loss: 8.061

 Epoch 8: 	Training loss: 13.166 	Validation loss: 7.405

 Epoch 9: 	Training loss: 12.543 	Validation loss: 7.455

 Epoch 10: 	Training loss: 12.424 	Validation loss: 7.263

 Epoch 11: 	Training loss: 11.758 	Validation loss: 7.511

 Epoch 12: 	Training loss: 12.340 	Validation loss: 7.253

 Epoch 13: 	Training loss: 11.682 	Validation loss: 7.240

 Epoch 14: 	Training loss: 12.260 	Validation loss: 7.325

 Epoch 15: 	Training loss: 11.602 	Validation loss: 7.029

 Epoch 16: 	Training loss: 11.281 	Validation loss: 7.296

 Epoch 17: 	Training loss: 11.737 	Validation l

### Question 8.3

In [22]:
print("Question 8.3:")

class MLP83(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.tanh(x)
        out = self.output(x)
        return out.squeeze(1)

model = MLP83(
    input_dims=X_train.shape[1], 
    hidden_dims=64, 
    output_dims=1
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

epochs = 100
train_losses, val_losses = [], []
train_r2, val_r2 = [], []

for epoch in range(epochs):
    train_loss = 0.0
    train_target, train_predict = [], []
    val_target, val_predict = [], []
    
    model.train()
    
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(X_samples)
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()

        train_loss += loss.item()
        
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))
    
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            
            outputs = model(X_samples)
            loss = criterion(outputs, y_samples)
            
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))
    print(f"\n Epoch {epoch + 1}: \tTraining loss: {train_loss:.3f} \tValidation loss: {val_loss:.3f}")
    
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device))
    test_set_r2 = r_squared(y_test, y_pred)
    print("Evaluation on test set")
    print(f"R2: {test_set_r2}")

Question 8.3:

 Epoch 1: 	Training loss: 300.393 	Validation loss: 16.904

 Epoch 2: 	Training loss: 17.776 	Validation loss: 13.384

 Epoch 3: 	Training loss: 15.651 	Validation loss: 11.331

 Epoch 4: 	Training loss: 14.712 	Validation loss: 7.728

 Epoch 5: 	Training loss: 14.013 	Validation loss: 8.078

 Epoch 6: 	Training loss: 12.617 	Validation loss: 7.035

 Epoch 7: 	Training loss: 12.241 	Validation loss: 12.890

 Epoch 8: 	Training loss: 11.020 	Validation loss: 6.790

 Epoch 9: 	Training loss: 11.681 	Validation loss: 7.783

 Epoch 10: 	Training loss: 10.476 	Validation loss: 7.573

 Epoch 11: 	Training loss: 10.037 	Validation loss: 7.166

 Epoch 12: 	Training loss: 10.924 	Validation loss: 7.408

 Epoch 13: 	Training loss: 9.410 	Validation loss: 6.846

 Epoch 14: 	Training loss: 9.591 	Validation loss: 5.411

 Epoch 15: 	Training loss: 10.093 	Validation loss: 7.237

 Epoch 16: 	Training loss: 9.192 	Validation loss: 6.043

 Epoch 17: 	Training loss: 9.894 	Validation los